In [1]:
from collections import namedtuple
import pathlib
import random

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
from orchid import (project_loader as opl)

In [4]:
import toolz.curried as toolz

In [5]:
import UnitsNet

In [6]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [7]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

{'bakken': WindowsPath('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac'),
 'montney': WindowsPath('c:/src/Orchid.IntegrationTestData/Project-frankNstein_Montney_UTM13_METERS.ifrac')}

In [8]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

{'bakken': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x00000159D3360340>,
 'montney': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x00000159D33603A0>}

In [9]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

{'bakken': <Orchid.FractureDiagnostics.UnitSystem object at 0x00000159D3345610>,
 'montney': <Orchid.FractureDiagnostics.UnitSystem object at 0x00000159D33451F0>}

In [10]:
projects['bakken'].WellTimeSeriesList.Items

<Orchid.FractureDiagnostics.TimeSeries.IWellSampledQuantityTimeSeries[] object at 0x00000159D3345430>

In [11]:
len(projects['bakken'].WellTimeSeriesList.Items)

4

In [12]:
first_well_time_series = projects['bakken'].WellTimeSeriesList.Items[0]
first_well_time_series

In [76]:
TimeSeriesSummary = namedtuple('TimeSeriesSummary', ['display_name', 'name',
                                                     'qty_name', 'qty_type', 'suffix', 
                                                     'start', 'stop',])

def summarize_time_series(ts):
    time_series_range = ts.GetTimeSeriesRange()
    start = time_series_range.Item1
    stop = time_series_range.Item2
    return TimeSeriesSummary(ts.DisplayName, ts.Name, 
                             ts.SampledQuantityName, ts.SampledQuantityType, ts.Suffix,
                             start, stop)

time_series_summary = summarize_time_series(first_well_time_series)
time_series_summary

TimeSeriesSummary(display_name='MonitorData-Demo_1H-Pressure', name='MonitorData-Demo_1H', qty_name='Pressure', qty_type=68, suffix='', start=<System.DateTime object at 0x00000159F4270490>, stop=<System.DateTime object at 0x00000159D13D7C10>)

In [14]:
first_well_time_series_samples = first_well_time_series.GetOrderedTimeSeriesHistory()
len(first_well_time_series_samples)
[first_well_time_series_samples[n].Value for n in range(5)]

[12.83973, 12.81262, 12.81063, 12.79626, 12.83592]

In [72]:
def generate_sample_indices(ts):
    candidate_indices = list(range(len(first_well_time_series_samples)))
    random.shuffle(candidate_indices)
    inner_indices = sorted(candidate_indices[:6])
    result = [0] + inner_indices + [len(first_well_time_series_samples) - 1]
    return result

In [73]:
sample_indices = generate_sample_indices(first_well_time_series_samples)
sample_indices

[0, 45415, 56552, 56601, 59190, 74283, 113370, 114594]

In [74]:
@toolz.curry
def make_sample_pressure(summary, sample):
    return UnitsNet.Pressure.From(UnitsNet.QuantityValue.op_Implicit(sample.Value), 
                                  project_units['bakken'].PressureUnit)

def sample_time_series(indices, summary, ts):
    result = toolz.pipe(indices,
                        # toolz.map(toolz.do(lambda n: print(n))),
                        toolz.map(lambda n: toolz.nth(n, ts)),
                        toolz.map(toolz.do(lambda s: print(s.Value))),
                        toolz.map(make_sample_pressure(summary)),
                        toolz.map(str),
                        list,
                        )
    return result
    
sample_time_series(sample_indices, time_series_summary, first_well_time_series_samples)

12.83973
8188.938
4479.67
3961.965
3021.48
4303.582
12.94474
-7999.0


['12.84 psi',
 '8,188.94 psi',
 '4,479.67 psi',
 '3,961.97 psi',
 '3,021.48 psi',
 '4,303.58 psi',
 '12.94 psi',
 '-7,999 psi']